# Multiple word anagram solver

In our group of friends, we have a tradition wherein every year someone organises a weekend away of which the destinations is a suprise. Hence the name 'surprise weekend'. There will be hints provided that give clues about the destination of the weekend. This time the hint was an email with the text:

In [ ]:
hint = "haag doen oer"

As nobody knew what to do with it, we got another hint saying it is an anagram. Needless to say, we use Python to solve the hint.

## Word lists

First thing we need is a list of Dutch words to match against. 

In this case, we took a list of 10.000 dutch words retrieved from another [Github project](https://github.com/mxsasha/python-zxcvbn) and saved it in the `data/` directory. 

In [ ]:
with open('../../data/dutch10000.txt', "r", encoding='latin-1') as file:
    wordlist = [word.strip() for word in file.readlines()] # remove \n
    
wordlist[0:5]

Not all words are in the vocabulaire. And one of the friends pointed out the word 'hoer' might be relevant, so let's add that one manually.

In [ ]:
wordlist.append('hoer')

### Places list

Another friend pointed out that we might need to match on place names as well. Thus, we find a website and prase a list of all Dutch placenames: https://www.metatopos.eu/almanak.html

In [ ]:
import urllib3
from bs4 import BeautifulSoup
from collections import OrderedDict

In [ ]:
http = urllib3.PoolManager()

# request the page and parse the html
page = http.request('GET', 'https://www.metatopos.eu/almanak.html')
soup = BeautifulSoup(page.data, 'html.parser')

# Take out the table records by their tag
records = soup.find_all('tr')

# get the place names
places = [record.find_all('td')[1].text for record in records]

# remove duplicates
places = list(OrderedDict.fromkeys(places))

print('Number of places:', len(places))

Now let's keep the list of names that start with a capital letter for pretty printing later on, and create another list with lowercases placenames for the matching.

In [ ]:
placelist = [place.lower() for place in places]

Adding `wordlist` and `placelist` to the total lookup list:

In [ ]:
lookuplist = wordlist + placelist

print('Number of lookup words:', len(lookuplist))

## Anagram or Scrabble solver?

To solve the puzzle we need something in between an Anagram finder and a Scrabble solver:
* **Anagram Finder**: Takes in a string and returns its anagrams using *all* letters from the string. For example `'dgo'` maps to `['dog', 'god']`.
* **Word Finder**: Finds all combinations of string of lengths 2, 3, .., n where n is the length of the string. Then uses the Anagram Finder to get words for those strings. This is more of the Scrabble approach wherein you don't intent to use all the letters on your board, but you are interested in finding all possible combinations.

Luckily for us, open source is great and we found the perfrect [scrabble repo](https://github.com/adrielklein/scrabble-word-finder) to use for our analysis. The `AnagramFinder` and `WordFinder` classes below come from this repository. Thanks!

In [ ]:
from collections import defaultdict

class AnagramFinder(object):
    def __init__(self, words):
        self._words = words
        self._alphagram_2_words()

    def _alphagram_2_words(self):
        """
        Builds dictionary with keys being alphagrams, and values being a list of words.
        
        I.e. {'een': {'een', 'ene', 'nee'}, ...}
        """
        self._result = defaultdict(lambda: set())  # save in class instead of in workzeug cache
        for word in self._words:
            alphagram = ''.join(sorted(word))
            self._result[alphagram].add(word)

    def get_anagrams(self, letter_string):
        alphagram = ''.join(sorted(letter_string))
        return sorted(list(self._result[alphagram]))

A demonstration of the `AnagramFinder`:

In [ ]:
anagram_finder = AnagramFinder(lookuplist)

print('een:', anagram_finder.get_anagrams(letter_string='een'))
print('wzolle:', anagram_finder.get_anagrams(letter_string='wzolle'))

Now let's look into the `WordFinder`:

In [ ]:
import itertools

def _get_combinations(letters_string):
    result = []
    for i in range(2, len(letters_string) + 1):
        for combination in itertools.combinations(letters_string, i):
            result.append(''.join(combination))
    return result


class WordFinder(object):
    def __init__(self, anagram_finder):
        self._anagram_finder = anagram_finder

    def get_words(self, letters_string):
        words = set()
        for combination in _get_combinations(letters_string):
            words.update(self._anagram_finder.get_anagrams(combination))
        return words

In [ ]:
word_finder = WordFinder(AnagramFinder(wordlist))

print('test:', word_finder.get_words('test'))

Perfect, now let's combine both and apply it to our hint.

In [ ]:
word_finder = WordFinder(AnagramFinder(wordlist))
len(word_finder.get_words(hint))

In [ ]:
list(word_finder.get_words(hint))[0:4]

Great so now we can create all possible combinations from a string. For the next part, we need to integrate this with the spaces and make sure we generate 3 words (as we have 2 spaces).

## Removing letters from string

To do this, we need to keep track of which letters from the hint we have already used and we cannot use anymore for the subsequent words. 

Let's first write a function to substract a single letter from a string and return the result:

In [ ]:
def remove_one_letter_from_string(letter, string):
    idx = string.find(letter)
    return string[:idx] + string[idx + 1:]

In [ ]:
remove_one_letter_from_string('o', hint)

Now write a function that substracts a string (consisting of multiple letters) form another string:

In [ ]:
def substract_string_from_string(string_to_remove, string_to_remove_from):
    result = string_to_remove_from
    for letter in string_to_remove:
        result = remove_one_letter_from_string(letter, result)
    return result

In [ ]:
substract_string_from_string('gr', hint)

Perfect. Now we can combine everything!

## Combine all and solve the hint!

To solve the hint we consider the following functional flow of our program:

1. Pick a first word
2. Apply a space and pick the second word. 
    * if not possible, go back to step 1 and try next word
3. Apply a space and pick the last word.
    * if not possible, go back to step 2 and try next word
    * if combination found -> print!
    
In each step, we keep track of the letters that remain so that we are sure that we have used all letters of the hint at the end.

In [ ]:
word_finder = WordFinder(AnagramFinder(lookuplist))

solutions = []  # save each solution
solutions_count = 0  # count possible solutions
solutions_places = []  # keep track of places used in a solution

# get first letters and word1 options
word1_letters = hint.replace(" ", "")
word1_options = word_finder.get_words(word1_letters)
for word1 in word1_options:
    # remove letters to create word2_letters
    word2_letters = substract_string_from_string(word1, word1_letters)
    # run the word_finder for word2_letters
    word2_options = word_finder.get_words(word2_letters)
    # is it possible to make a word 3, if yes continue
    if len(word2_options) > 0:
        for word2 in word2_options:
            # remove letters to create word3_letters
            word3_letters = substract_string_from_string(word2, word2_letters)
            # run the word_finder for word2_letters
            word3_options = word_finder.get_words(word3_letters)
            # check if word 3 found 
            if len(word3_options) > 0:
                for word3 in word3_options:
                    # AND check if all letters are exhausted:
                    if substract_string_from_string(word3, word3_letters) == '':
                        solution = ' '.join([word1, word2, word3])
                        # AND check if one of the words comes from a place:
                        if any(place in [word1, word2, word3] for place in placelist):
                            place1 = places[placelist.index(word1)] if word1 in placelist else ''
                            place2 = places[placelist.index(word2)] if word2 in placelist else ''
                            place3 = places[placelist.index(word3)] if word3 in placelist else ''
                            solutions_count += 1
                            solutions.append(solution)
                            solutions_places = solutions_places + [place1, place2, place3]
                            print(solutions_count, '- places:', [place1, place2, place3], '-', solution)


The solution must be somewhere in the above list! Meaning the hint is about one of the following places:

In [ ]:
# deduplicate places and remove empty place ''
solutions_places = list(OrderedDict.fromkeys(solutions_places))
solutions_places.remove('')
print('Mentioned places:', solutions_places)

Interesting options:

* 'naar hoog Ede'
* 'gaan door Hee'

Now let's see if we go to either one of those places..

Done!